In [3]:
%%writefile C:\Anaconda\Lib\CAMP.py
    
import datetime as dt
import pandas as pd
import numpy as np
import math as math 
import MathsUtilities as MUte
from scipy.optimize import curve_fit

CampConstants = {
    'k':-0.17,
    'VernalisationThreshold':1.0,
    'MethalationThreshold':0.5,
}

def LARPTQ(PTQ,maxLAR,minLAR,PTQhf):
    """Calculate the Leaf Appearance Rate at a given PTQ for a genotype of given minLAR and MaxLAR as formualted by Baumont etal 2019 Journal Expt Botany, Equation 3. 
    Args:
        PTQ: PhotoThermal Quatent (mmol PAR m-2 oCd-1)
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Leaf Appearance Rate (dHS/oCd)
    """
    return minLAR+((maxLAR-minLAR)*PTQ)/ (PTQhf+PTQ)

def CalcPTQ(Tt,Pp,lightIntensity):
    """Calculate the Photothermal Quotient for a given day in controlled environment (mmol PAR m-2 oCd-1). 
    Args:
        Tt: Mean daily thermal time (oC)
        Pp: Photoperiod (oCd-1)
        LightIntensity: radiation output when lights are on (MJ/m2/s)
    Returns:
        Photothermal Quotient (mmol PAR m-2 oCd-1)
    """
    Radn = Pp * 3600 * lightIntensity * 2  # Hours * s/h * MJ/m2/s * Coeff_PAR->Total 
    Ft = 1
    if Tt < 15:
        Ft = Tt/15
    return (Radn * 2.285 * Ft)/Tt

def CalcdHS(Tt,HS,PTQ,maxLAR,minLAR,PTQhf):
    """Calculate daily increase in Haun stage
    Args:
        Tt: Thermal time increment
        HS: Current Haun Stage
        PTQ: PhotoThermalQuotient
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Daily increas in Haun Stage
    """
    BasePhyllochron = 1/LARPTQ(PTQ,maxLAR,minLAR,PTQhf)
    StageFactor = np.interp(HS,[0,7,8,12],[1.0,1.0,1.4,1.4],False)
    Phyllochron = BasePhyllochron*StageFactor
    return Tt/Phyllochron

def CalcPpResponse(Pp, Base, Max, dHS):
    """ Calculate value of a photoperiod sensitive variable between Max and Baee values
    Args:
        Pp: Photoperiod
        Base: value below 8h Pp
        Max: value above 16h Pp
        dHS: delta haun stage
    Returns:
        variable value dependent on photoperiod and adjusted for dHS
    """
    if Pp <= 8.0:
        return Base * dHS
    if (Pp > 8.0) and (Pp < 16.0):
        return (Base + (Max * (Pp-8)/(16-8))) * dHS
    if (Pp >= 16.0):
        return Max * dHS  

def CalcBaseUpRegVrn1(Tt, dHS, BaseDVrn1):
    """ Calculate upregulation of base Vrn1
    
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        BaseDVrn1: coeffociennt for Base Vrn1 expression
        
    Returns:
        delta BaseVrn1 representing the additional Vrn1 expression from base expression
    """
    if Tt < 0: 
        BaseDVrn1 = 0
    return BaseDVrn1 * dHS

def CalcColdUpRegVrn1(Tt,dHS, MaxDVrn1, k):
    """ Upregulation of Vrn1 from cold.  Is additional to base vrn1
        BaseDVrn1 in seperate calculation otherwise te same as Brown etal 2013
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        MaxDVrn1: coefficient for Maximum upregulation of Vrn1
        k: The exponential coefficient determining temperature response
        
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    UdVrn1 = MaxDVrn1 * np.exp(k*Tt)
    if (Tt < 20):
        return UdVrn1 * dHS
    else:
        return -5
    
def TSHS(FLN, IntFLNvsTSHS):
    """Haun stage timing of terminal spikelet 
       Inverts equation 5 from Brown etal 2013 FLN =  2.85 + 1.1*TSHS
       Note the intercept differs, was typeo on publication
       Intercept has been made variable for as needs to be lower for some very fast varieties
    Args:
        FLN: The final leaf number
        IntFLNvsTSHS: The intercept of the regression between FLN and TSHS
    Returns:
        Estimation of the haun stage timing of terminal spikelet
    """
    return (FLN - IntFLNvsTSHS)/1.1

def CAMPmodel(Out, Day, Tt, Pp, Params, Consts, TtEmerge, LightIntensity):
    """ The Cereal Anthesis Molecular Phenology model.
        Based on the ideas presented in Brown etal 2014 (Annals of Botany)
        Alterations made replacing Vrn4 notion with methalation of Vrn1, introducing VrnX to account for long day vernalisation
        Vrn2 approach changed to have hihger experssion for first HS then a lower rate
        Other alterations to implement working code base
    Args:
        Out: "FLN" or None.  If "FLN" will only return estimated FLN else will return full dataframe with daily state variable values
        Day: List, 1:EndDay representing the timesteps in model run
        Tt: List of same length as Day, representing daily temperature
        Pp: List of same length as Day, representing daily photoperiod
        Params: Dict with genotype parameters fitted by CalcCultivarVrnCoeffs() 
        Consts: Dict with crop specific constants,
        TtEmerge: Thermaltime from sowing to emergence
        LightIntensity: radiation intensity (MJ/m2/s)
    """
    # Set up Data structure and initialise values
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    DF = pd.DataFrame(index = [0], columns = ['Day','Tt','Pp'])
    IsGerminated = False
    IsEmerged = False
    IsDayOfEmergence = False
    IsMethalating = False
    IsVernalised = False
    IsInduced = False
    IsReproductive = False
    IsAtFlagLeaf = False
    DF.loc[0,'Day'] = 0
    DF.loc[0,'Tt'] = Tt[0]
    DF.loc[0,'Pp'] = Pp[0]
    DF.loc[0,'PTQ'] = 0
    DF.loc[0,'Stage'] = ""
    DF.loc[0,'HS'] = 0.0
    DF.loc[0,'AccumTt'] = 0
    DF.loc[0,'BaseVrn1'] = 0
    DF.loc[0,'ColdVrn1'] = 0
    DF.loc[0,'MethColdVrn1'] = 0
    DF.loc[0,'pVrn2'] = 0
    DF.loc[0,'Vrn1'] = 0
    DF.loc[0,'Vrn2'] = 0.0
    DF.loc[0,'Vrn3'] = 0.0
    DF.loc[0,'VrnX'] = 0.0
    DF.loc[0,'VSHS'] = 0
    DF.loc[0,'FIHS'] = 0
    DF.loc[0,'TSHS'] = 0
    DF.loc[0,'FLN'] = Params.IntFLNvsTSHS
    DF.loc[0,'dHS'] = 0
    DF.loc[0,'dBaseVrn1'] = 0
    DF.loc[0,'dColdVrn1'] = 0
    DF.loc[0,'dMethColdVrn1'] = 0
    DF.loc[0,'dpVrn2'] = 0
    DF.loc[0,'dVrn3'] = 0
    DF.loc[0,'dVrnX'] = 0
    DF.loc[0,'HSEmerge'] = 0
    d = 1
    BaseVrn1AtEmergence = 0
    # Model daily loop
    # ^^^^^^^^^^^^^^^^
    while (IsAtFlagLeaf == False) and (d < Day[-1]):
        #Sow crop on day 1
        DF.loc[d,'Day'] = d
        if (IsGerminated == False):
            DF.loc[d,'Stage'] = 'Germination'
            IsGerminated = True
        # Set daily environment variables
        DF.loc[d,'Tt'] = Tt[d-1]
        DF.loc[d,'Pp'] = Pp[d-1]
        #Zero set deltas
        DF.loc[d,'dBaseVrn1'] = 0
        DF.loc[d,'dColdVrn1'] = 0
        DF.loc[d,'dMethColdVrn1'] = 0
        DF.loc[d,'dpVrn2'] = 0
        DF.loc[d,'dVrn3'] = 0
        DF.loc[d,'dVrnX'] = 0
                
        DF.loc[d,'AccumTt'] = DF.loc[d-1,'AccumTt'] + DF.loc[d,'Tt']
        
        #PropnOfDay = 1.0
        if (DF.loc[d,'AccumTt'] > TtEmerge) and (IsEmerged==False):
            IsEmerged = True
            IsDayOfEmergence = True
            DF.loc[d,'Stage'] = 'Emergence'
            #PropnOfDay = (DF.loc[d,'AccumTt'] - TtEmerge)/DF.loc[d,'Tt'] # Calculate fraction of emergence days Tt that is not used for emergence
        
    # Calculate daily Haun Stage changes
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        DF.loc[d,'PTQ'] = CalcPTQ(DF.loc[d,'Tt'],DF.loc[d,'Pp'],LightIntensity)
        if IsEmerged==True: # Crop emerged
            # Calculate delta haun stage
            DF.loc[d,'dHS'] = CalcdHS(DF.loc[d,'Tt'],DF.loc[d-1,'HS'],DF.loc[d,'PTQ'],Params.maxLAR,Params.minLAR,Params.PTQhf)
        else: # Crop not yet emerged but we still need a dHS value to drive Vrn1 expression prior to emergence
            EmergDurationFactor = 1
            if (DF.loc[d,'AccumTt'] > 90):  # Calculate EmergenceDurationFactor to slow accumulation of HS if emergence is taking a long time.  This slows Vrn1 expression under slow emergence and strange responses to delayed sowing
                EmergDurationFactor = 1 * np.exp(-0.015 * (DF.loc[d,'AccumTt']-90))
            DF.loc[d,'dHS']  = DF.loc[d,'Tt']/(1/LARPTQ(1.0,Params.maxLAR,Params.minLAR,Params.PTQhf)) * EmergDurationFactor
            DF.loc[d,'HSEmerge'] = DF.loc[d-1,'HSEmerge'] + DF.loc[d,'dHS'] #* PropnOfDay
       
        #increment HS
        if IsEmerged == True:
            DF.loc[d,'HS'] = DF.loc[d-1,'HS'] +  DF.loc[d,'dHS']
        else:
            DF.loc[d,'HS'] = 0
    
    # Calculate Vrn gene expression
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        if (IsVernalised==False): # do vernalisation calculations if crop not yet vernalised
        # Calculate Vrn1 deltas
            DF.loc[d,'dBaseVrn1'] = CalcBaseUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.BaseDVrn1) 
            DF.loc[d,'dColdVrn1'] = CalcColdUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.MaxDVrn1,Consts['k'])
            DF.loc[d,'ColdVrn1'] = max(0.0,DF.loc[d-1,'ColdVrn1'] + DF.loc[d,'dColdVrn1'])
            
        # Calculate daily methalation
            if DF.loc[d,'ColdVrn1']>= Consts['MethalationThreshold']: # ColdVrn1 expressed to threshold required for methalation to occur
                IsMethalating = True
            else:
                IsMethalating = False
                
            if (IsMethalating == True):
                DF.loc[d,'dMethColdVrn1'] = min(DF.loc[d,'ColdVrn1'] - Consts['MethalationThreshold'],
                                                max(0,DF.loc[d,'dColdVrn1']))
            
        # Calcualte expression of photoperiod sensitive genes
            DF.loc[d,'dVrnX'] = 0.0
            DF.loc[d,'dpVrn2'] = 0.0
            if (IsEmerged==True):  # Photoperiod sensitive genes only express after emergence
                if DF.loc[d-1,'MethColdVrn1'] == 0.0:  # VrnX expression only occurs if no methalation of Vrn1 has occured
                    DF.loc[d,'dVrnX'] = CalcPpResponse(DF.loc[d,'Pp'], 0.0, Params.MaxDVrnX, DF.loc[d,'dHS'])
                
                if IsDayOfEmergence == True: # Calculate high dpVrn2 on emergence day to give initial value
                    BaseVrn1AtEmergence = DF.loc[d-1,'BaseVrn1'] + DF.loc[d,'dBaseVrn1']
                    DF.loc[d,'dpVrn2'] = CalcPpResponse(DF.loc[d,'Pp'],BaseVrn1AtEmergence,Params.MaxIpVrn2,1.0)
                else:
                    DF.loc[d,'dpVrn2'] = CalcPpResponse(DF.loc[d,'Pp'],0,Params.MaxDpVrn2,DF.loc[d,'dHS'])
                    
                
        # Increment todays Vrn expression values using deltas just calculated
            DF.loc[d,'pVrn2'] = DF.loc[d-1,'pVrn2'] + DF.loc[d,'dpVrn2']
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX'] + DF.loc[d,'dVrnX']
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1'] + DF.loc[d,'dBaseVrn1']
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] + DF.loc[d,'dMethColdVrn1'] 
            # Effective expression of Vrn1 is the sum of baseVrn1, MethalatedVrn1 and Vrnx expression
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1'] + DF.loc[d,'dMethColdVrn1'] \
                                                          + DF.loc[d,'dBaseVrn1'] \
                                                          + DF.loc[d,'dVrnX'] 
            # Effective Vrn2 expression is the potential expression less that which is blocked by Vrn1
            DF.loc[d,'Vrn2'] =  max(0.0,DF.loc[d,'pVrn2'] - DF.loc[d,'Vrn1'])
                                    
        # Once Vernalisation complete, carry over vern expression values from yesterday
        else:
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1']
            DF.loc[d,'ColdVrn1'] = DF.loc[d-1,'ColdVrn1'] 
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] 
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1']
            DF.loc[d,'pVrn2'] = DF.loc[d-1,'pVrn2']
            DF.loc[d,'Vrn2'] = DF.loc[d-1,'Vrn2']
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX']
            
    # Set Haun stage variables
    # ^^^^^^^^^^^^^^^^^^^^^^^^
        if IsVernalised == False:
            DF.loc[d,'VSHS'] = DF.loc[d,'HS']
        else:
            DF.loc[d,'VSHS'] = DF.loc[d-1,'VSHS']
        if IsInduced == False:
            DF.loc[d,'FIHS'] = DF.loc[d,'HS']
        else: 
            DF.loc[d,'FIHS'] = DF.loc[d-1,'FIHS']
        if IsReproductive == False:
            DF.loc[d,'TSHS'] = DF.loc[d,'HS']
            DF.loc[d,'FLN'] = Params.IntFLNvsTSHS + 1.1 * DF.loc[d,'TSHS']
        else:
            DF.loc[d,'TSHS'] = DF.loc[d-1,'TSHS']
            DF.loc[d,'FLN'] = DF.loc[d-1,'FLN']

    # Work out if vernalisation is complete    
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Vernalisation saturation occurs when Vrn1 > the vernalisation threshold and Vrn2 expression is zero
        if (IsEmerged==True) and (DF.loc[d,'Vrn1'] >= Consts['VernalisationThreshold']) and (DF.loc[d,'Vrn2'] == 0) and (IsVernalised==False):
            IsVernalised  = True
            DF.loc[d,'Stage'] = 'Vern Sat'
        
    # Then work out Vrn3 expression
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Vrn3 expression occurs between Vernalisation saturation and Terminal spikelet
        if (IsVernalised == True) and (IsReproductive == False):
            DF.loc[d,'dVrn3'] = CalcPpResponse(DF.loc[d,'Pp'], Params.BaseDVrn3, Params.MaxDVrn3, DF.loc[d,'dHS'])
        DF.loc[d,'Vrn3'] = min(1,DF.loc[d-1,'Vrn3'] + DF.loc[d,'dVrn3'])

    # Finally determine phenological stage
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Then work out phase progression based on Vrn expression
        if (DF.loc[d,'Vrn3'] >= 0.3) and (IsInduced == False):
            IsInduced = True;
        if (DF.loc[d,'Vrn3'] >=  1.0) and (IsReproductive == False):
            IsReproductive = True;
            DF.loc[d,'Stage'] = 'Term Spike'
        
        #Work out if Flag leaf has appeared.
        if DF.loc[d,'HS'] >= DF.loc[d,'FLN']:
            IsAtFlagLeaf = True    
            DF.loc[d,'Stage'] = 'FlagLeaf'
    
        #Add states to dataframe
        IsDayOfEmergence = False
        
        DF.loc[d,'IsGerminated'] = IsGerminated
        DF.loc[d,'IsEmerged'] = IsEmerged
        DF.loc[d,'IsMethalating'] = IsMethalating
        DF.loc[d,'IsVernalised'] = IsVernalised
        DF.loc[d,'IsReproductive'] = IsReproductive
        DF.loc[d,'IsAtFlagLeaf'] = IsAtFlagLeaf
    
        # Increment day
        d += 1
        
    if (Out == 'FLN') and (IsAtFlagLeaf == True):
        return DF.iloc[-1,:]['FLN']
    else:
        return DF
    
def CalcCultivarVrnCoeffs(inputs):
    """ Calculate VrnRate paremeters for genotype from given FLN observations under set vernalisation and photoperiod treatments
    Args:
        inputs: Dict or Series containing the following.
        FLN_LV: Final Leaf Number under long day full vernalisation treatment
        FLN_LN: Final Leaf Number under long day un-vernalisation treatment
        FLN_SV: Final Leaf Number under short day full vernalisation treatment
        FLN_SN: Final Leaf Number under short day un-vernalisation treatment
        VrnTreatTemp: Vernalisation Treatment Temperature
        VrnTreatDuration: Days exposure to vernalisting temperature
        TreatmentPTQ_L: The PTQ under long Pp,
        TreatmentPTQ_S: The PTQ under short Pp,
        TtEmerge: Observed Thermal time from sowing to emergence
        k: Vernalisation temperature response coefficient
        VernSatThreshold: The amount of Methalated Vrn1 that is required for vernilisation saturation to occur
        MethVrn1Threshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Series with values assigned to each variable in CultivarParameterList 
    """
    #Set up Data Store
    data = pd.Series()
    
    # copy some params into outputs for checking
    data['maxLAR'] = inputs['maxLAR']
    data['minLAR'] = inputs['minLAR']
    data['PTQhf']  = inputs['PTQhf']
    data['FLN_LV'] = inputs['FLN_LV']
    data['FLN_LN'] = inputs['FLN_LN']
    data['FLN_SV'] = inputs['FLN_SV']
    data['FLN_SN'] = inputs['FLN_SN']
    
###########################################
# Calculate phase durations (in Haun Stage)
###########################################
    
    # Haun stage duration of Emergence (EmergHS), assume a PTQ of 1.0 suitable for before emergence
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['EmergHS'] = inputs['TtEmerge']/(1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
    
    # Haun stage duration of vernalisation treatment period
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # If Vern treatment complete before emergence (VernTreatHS_L,VernTreatHS_S), assume a PTQ of 1.0 suitable for before emergence
    data['VrnTreatTtDurat'] = inputs['VrnTreatDuration'] * inputs['VrnTreatTemp']
    if data['VrnTreatTtDurat'] <= inputs['TtEmerge']: 
        data['VernTreatHS_L'] = data['VrnTreatTtDurat']/\
                                (1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
        data['VernTreatHS_S'] = data['VrnTreatTtDurat']/\
                                (1/LARPTQ(1.0,inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf']))
    # If Vern treatment completes after emergence, add the HS when treatment finished to Emergence HS
    else: 
        data['EmergToEndTreatHS_L'] = (data['VrnTreatTtDurat'] - inputs['TtEmerge'])/\
                                      (1/LARPTQ(inputs['TreatmentPTQ_L'],inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf'])) 
        data['EmergToEndTreatHS_S'] = (data['VrnTreatTtDurat'] - inputs['TtEmerge'])/\
                                      (1/LARPTQ(inputs['TreatmentPTQ_S'],inputs['maxLAR'],inputs['minLAR'],inputs['PTQhf'])) 
        data['VernTreatHS_L'] = data['EmergHS'] + data['EmergToEndTreatHS_L']
        data['VernTreatHS_S'] = data['EmergHS'] + data['EmergToEndTreatHS_S']
    
    # The soonest a plant may exhibit vern saturation if vernalisation completed early
    MinVSHS = 1.1
    
    # Minimum Haun stage duration from vernalisation saturation to terminal spikelet under long day conditions (MinHSVsTs)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # Assume maximum of 3, Data from Lincoln CE (CRWT153) showed varieties that harve a high TSHS hit VS ~3HS prior to TS 
    data['MinHSVsTs'] = min(3,(data['FLN_LV'] - MinVSHS)/2)
    MinVSHS_FLN_LV = data['FLN_LV'] - MinVSHS - data['MinHSVsTs']
    data['IntFLNvsTSHS'] = min(2.85,MinVSHS_FLN_LV/1.1)
    
    # Calculate Terminal spikelet duration (TSHS) for each treatment from FLNData
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['TSHS_LV'] = TSHS( data['FLN_LV'], data['IntFLNvsTSHS'])
    data['TSHS_LN'] = TSHS( data['FLN_LN'], data['IntFLNvsTSHS'])
    data['TSHS_SV'] = TSHS( data['FLN_SV'], data['IntFLNvsTSHS'])
    data['TSHS_SN'] = TSHS( data['FLN_SN'], data['IntFLNvsTSHS'])
        
    # Photoperiod sensitivity (PPS) is the difference between TSHS at 8 and 16 h pp under full vernalisation.
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['PPS'] = max(0,data['TSHS_SV'] - data['TSHS_LV'])
    
    # Vernalisation Saturation duration (VSHS) for each environment from TSHS and photoperiod response
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # Terminal spikelet duration less the minimum duration from VS to TS under long day treatment
    data['VSHS_LV'] = max(MinVSHS, data['TSHS_LV'] - data['MinHSVsTs']) 
    data['VSHS_LN'] = max(MinVSHS, data['TSHS_LN'] - data['MinHSVsTs']) 
    # Terminal spikelet duration less the minimum duration from VS to TS and the photoperiod extension of VS to TS under short day treatment
    data['VSHS_SV'] = max(MinVSHS, data['TSHS_SV'] - (data['MinHSVsTs'] + data['PPS']))
    data['VSHS_SN'] = max(MinVSHS, data['TSHS_SN'] - (data['MinHSVsTs'] + data['PPS']))
    
####################################
# Calculate Photoperiod sensitivities
####################################
    
    # Maximum delta for Upregulation of Vrn3 (MaxDVrn3)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # Occurs under long Pp conditions, Assuming Vrn3 increases from 0 - VernSatThreshold between VS to TS and this takes 
    # MinHSVsTs under long Pp conditions.
    data['MaxDVrn3'] = inputs['VernalisationThreshold'] / data['MinHSVsTs']
    
        
    # Base delta for upredulation of Vrn3 (BaseDVrn3)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # Occurs under short Pp conditions, Assuming Vrn3 increases from 0 - VernSatThreshold from VS to TS 
    # and this take 3 HS plus the additional HS from short Pp delay.
    data['BaseDVrn3'] = inputs['VernalisationThreshold'] / (data['MinHSVsTs'] + data['PPS'])
    
    ## Under long day conditions Vrn3 expresses at its maximum rate and plant moves quickley from VS to TS.
    ## Genotypic variation in MaxDVrn3 will contribute to differences in earlyness per se
    ## Under shord day condition Vrn3 expressssion may be slower, taking longer to get from VS to TS
    ## Photoperiod sensitiviey of a genotype is determined by differences in BaseVrn3 and MaxVrn3.  
    ## if the two values are the same the genotype will not show photoperiod sensitivity
    ## the greater the difference the more resonse the genotype will show to photoperiod
    
#################################
# Calculate Base development rate 
#################################

    # Base delta for upregulation of Vrn1 (BaseDVrn1) 
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    # The rate of expression is calculated as the amount of expression divided by the duration
    # Under short Pp treatment Vrn2 is absent so the amount of Vrn1 expression required for VS to occur is given by VernalisationThreshold
    # Under non vernalising conditions Vrn1 expression will happen at the base rate.  
    # VrnX expression is absent in short days so baseVrn1 is the only contributor to the timing of VS
    # BaseVrn1 expression starts when the seed is imbibed so the duration of expression is emergence plus VS
    data['BaseDVrn1'] = inputs['VernalisationThreshold'] / (data['VSHS_SN'] + data['EmergHS'])
    
    # Genotypic variation in BaseDVrn1 contributes to intrinsic earlyness.
    # A genotype with high BaseDVrn1 will reach VS quickly regardless of vernalisation exposure
    # A genotype with low BaseDVrn1 will reach VS slowly if not vernalised but the duration of VS may be decreased
    # by exposure to vernalising temperatures depending on cold vernalisation sensitivity
    
#############################################
# Cold Vernalisation Sensitivity Calculations
#############################################
    
    # Calcualtions of vernalisation sensitivity use data from short Pp treatments because Vrn2 and Vrnx are absent
    # in these conditions and measured vernalistion response will be the result of Vrn1 expression alone.
    
    # Vernalisation Sensitivity (VS) measured simply as the difference between SV and SN treatuemnts
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['VS'] = data['VSHS_SN'] - data['VSHS_SV'] 
    
    # To determine the effects of vernalisation treatment on Vrn1 expression we need to seperate these from baseVrn1 expression
    # BaseVrn1 expression at VSHS ('BaseVrn1AtVSHS_SV')
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['BaseVrn1AtVSHS_SV'] = (data['VSHS_SV']+data['EmergHS']) * data['BaseDVrn1']
    
    # Any accelleration in VS due to cold exposure under short Photoperiod will be due to methatalated Vrn1 expression
    # The amount of Vrn1 required for VS is given by the Vernalisation Threshold so the amount of methalated Vrn expression
    # at VS must be this threshold less the amount of vrn1 contributed by base Vrn1 expression over this duration
    # Methalated Cold Vrn1 expression under short Pp vernalisiation treatment (MethColdVern1AtTrans_SV)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['MethColdVern1AtTrans_SV'] = max(0,inputs['VernalisationThreshold'] - data['BaseVrn1AtVSHS_SV'])
    
    # Cold Vrn 1 expression must first be upregulated to a methalation threshold and any further expression beyond this 
    # threshold is methalated into a persistant vernalisation response.  Thus the total expression of cold Vrn1 due to
    # the vernalisation treatment applied is givin by MethColdVern1AtTrans_SV plus the Methalation Threshold
    # Cold Vern1 expression at VSHS under short Pp vernalised conditions (ColdVrn1AtVSHS_SV)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['ColdVrn1AtVSHS_SV'] = inputs['MethalationThreshold'] + data['MethColdVern1AtTrans_SV']
    
    # The haun stage duration over which vernalisation temperatures will have an effect is the minimum of the cold 
    # treatment duration (VernTreatHS) and the HS when vernalisation occurs as cold exposure after this is irrelevent
    # Effective HS duration of cold treatment (EffectiveColdHS)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['EffectiveColdHS'] = min(data['VSHS_SV']+data['EmergHS'], data['VernTreatHS_S']);
    
    # Then we calculate the rate as the amount divided by the duration
    # Rate of cold Vrn1 expression at the vernalisation treatment temperture ('DVrn1AtVrnTreatTemp')
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['DVrn1AtVrnTreatTemp'] = data['ColdVrn1AtVSHS_SV']/data['EffectiveColdHS']
    
    # This rate is dependent on the temperature of the duration treatment and can be extrapolated to the maximum rate (at 0oC)
    # using the exponential funciton proposed by Brown etal 2013 Annals of Botany.
    # dVrn1 = MaxdVrn1 * np.exp(k*VrnTreatTemp) as DVrn1 At VrnTreatTemp' is known 
    # MaxDVrn1 is set to vero for genotypes with VS < 0.5 as these varieties are insensitive and calculated rates of MaxDVrn1 are simply amplifying noise
    # Maximum upregulation delta Vrn1 (MaxDVrn1)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    if data['VS']  > 0:
        data['MaxDVrn1'] = data['DVrn1AtVrnTreatTemp']/ np.exp(inputs['k'] * inputs['VrnTreatTemp'])
    else:
        data['MaxDVrn1'] = 0.0
    
    # Genotypic variation in MaxDVrn1 combine with BaseDVrn1 to determine cold temperature vernalisatin sensitiviy
    # A genotype with low BaseDVrn1 and high MaxDVrn1 will show high vernalisation sensitivity and sensitivity will decline 
    # either BaseDVrn1 increasees of MaxDVrn1 decreases.
                                
######################################
# Photoperiod effects on vernalisation
######################################
    
    # The parameters calculated above deal with photoperiod sensitivity in fully vernalised crops and vernalisation sensitivity
    # under short photoperiod where cold is the only factor driving vernalisation response.
    # Under long days photoperiod can also interact with vernalisation, either slowing the rate of vernalisation or speding it up
    
    # Vernalisation photoperiod sensitivy parameter (VPPS) is calculated to determine what effect photoperiod will have.
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['VPPS'] = 1-((data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    
    # Genotypes that have a negative VPPS demonstrate short day vernalisation.  Under long days vernalisation requirement is reduced
    # This is caused by the expression of Vrn2 which must be blocked by additional expression of Vrn1 meaning vernalisation takes longer.  
    # The assumed mechanisum for Vrn2 is that it is expressed to a potential level and that one unit of Vrn1 will block the effective 
    # expression of Vrn2 so actual Vrn2 expression will always be lower than potential
    # We can make some assumptions about the amounts of Vrn2 expression under long Pp conditions to calcualte rates                                
    # Firstly, under un-vernalised conditions we can calculate the potential Vrn2 expression simply from
    # the amount of BaseVrn1 expression up to VSHS
                                
    # Potential Vern2 expression at VS under long Pp UnVerrnalised treatment (pVrn2AtVSHS_LV)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['pVrn2AtVSHS_LN'] = (data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1']
                                
    # Under full vernalised treatment Vrn2 expression must be less than or equal to VernalisationThreshold at the time of VS.
    # If we assume it is equal to VernalisationThreshold this provides us with another amount of pVrn2
    # Potential Vern2 expression at VS under long Pp Vernalised treatment (pVrn2AtVSHS_LV)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['pVrn2AtVSHS_LV'] = inputs['VernalisationThreshold']
    
    # If we regress these two potential Vrn2 amounts against their durations to VS the slope give us a rate of Vrn2 expression
    # The resulting rate needs to always be less than BaseDVrn1 so Vrn1 can catch up with Vrn2 to cause vernalisation
    # the amount of Vrn2 needs to quickly exceed BaseDVrn1 soon after emergence to achieve a delay in VS.
    # The intercept of the regression quantifys how much Vrn2 would be expressed at emergence.  

    # Initial potential Vrn2 at emergence (IPVrn2) and potential Vrn2 rate there after (DpVrn2)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['MaxIpVrn2'] = 0
    data['MaxDpVrn2'] = 0 
    #Vrn2 parameters only relevent for genotypes with nevaitve VPPS and a low BaseDVrn1.  Genotypes with a high base
    if (data['VPPS'] < 0): #and (data['BaseDVrn1'] < 0.4): 
        if data['VSHS_LN']-data['VSHS_LV'] == 0:
            data['MaxIpVrn2'] = data['pVrn2AtVSHS_LN']
        else:
            data['MaxDpVrn2'] = max(0,(data['pVrn2AtVSHS_LN']-data['pVrn2AtVSHS_LV'])/(data['VSHS_LN']-data['VSHS_LV']))
            data['MaxIpVrn2'] = (data['pVrn2AtVSHS_LV'])-(data['VSHS_LV']*data['MaxDpVrn2'])
            if (data['MaxDpVrn2'] >= (data['BaseDVrn1']*0.99)): #If DpVrn2 exceeds baseVrn1 we need to do some forcing so it doesn't
                data['BaseVrn1AtVSHS_LV'] = (data['VSHS_LV'] + data['EmergHS']) * data['BaseDVrn1']
                data['MaxDpVrn2'] = max(0,(data['pVrn2AtVSHS_LN']-(data['BaseVrn1AtVSHS_LV']*1.2))/(data['VSHS_LN']-data['VSHS_LV']))
                data['MaxIpVrn2'] = (data['pVrn2AtVSHS_LN'])-(data['VSHS_LN']*data['MaxDpVrn2'])
                              
    # Genotypes that have a positive VPPS show an acelleration of vernalisation under long day conditions.
    # The molecular mechanium for this is uncertain so we attrubute it to VrnX
    # Under Long Pp unvernalised varieties will reach VS HS sooner than BaseVrn1 expresion determines.
    # BaseVrn1 expression at VS under long Pp unvernalised treatment (BaseVrn1AtVSHS_LN)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['BaseVrn1AtVSHS_LN'] = min(inputs['VernalisationThreshold'],(data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    
    # If we assume the expression of VrnX upregulates Vrn1 an equivelent amount then the amount of VrnX expression can be
    # Estimated as the difference between Base Vrn1 at VS and the Vernalisation threshold.  The duration of expression is
    # VSHS assuming VrnX is expressed from emergence until VS.
    # Maximum rate of Vrnx Expression (MaxDVrnX)
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    data['MaxDVrnX'] = (inputs['VernalisationThreshold']-data['BaseVrn1AtVSHS_LN'])/(data['VSHS_LN'])
    
    return data     

Overwriting C:\Anaconda\Lib\CAMP.py
